This notebook makes a dataframe for inputs (target and feature variables) from the Large Ensemble Testbed (LET) based on SOCAT and Argo observations and then reconstructs pCO2-Residual globally using XGB. See Zenodo link in the repository to access the sampling masks. 

First, the direct effect of temperature is removed from pCO2 (i.e., pCO2-T), giving the resulting pCO2-Residual which is used for the reconstruction. The code for this calculation is found in a different notebook in the repository ("calculate_pco2_components"). 

Section I:
Creates a dataframe for inputs (target and feature variables) from the Large Ensemble Testbed for the XGB algorithm. 

Section II:
Then, the XGBoost algorithm reconstructs the surface ocean pCO2-Residual globally over 1982-2016. 

Section III:
Finally, pCO2-T is added back to the pCO2-Residual to get pCO2.  

This notebook was originally created by Val Bennington, and modified by Thea Hatlen Heimdal and Devan Samant.

In [1]:
# standard imports
import os
import datetime
from pathlib import Path
from collections import defaultdict
import scipy
import random
import numpy as np
import xarray as xr
import pandas as pd
import joblib
import pickle

# machine learning libraries
import sklearn            # machine-learning libary with many algorithms implemented
import xgboost as xgb     # extreme gradient boosting (XGB)
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

# Python file with supporting functions
import pre_argo

### <font color='red'>Section I</font> 
#### Make dataframe for inputs for XGB algorithm

In [3]:
# Accessing directories

root_dir = "/data/artemis/workspace/theimdal/GO-BGC"

# Path to the dataframe created in this notebook
data_output_dir = f"{root_dir}/LET_selected_XGB/500_floats_optimized"

# Path to pickle file with list of LET members
reference_output_dir = "/data/artemis/workspace/theimdal/saildrone/LET_pickle_files"

# Path to "raw" LET output (testbed truth)
ensemble_dir_head = "/local/data/artemis/simulations/LET" 

In [4]:
# Loading references to "pick" members from the LET
# We use 75 members in total, 25 each from CESM, CanESM2 and GFDL
# "mems_dict" is a list of the members of the LET

path_LET = f"{reference_output_dir}/members_LET_dict.pickle"
path_seeds = f"{reference_output_dir}/random_seeds.npy"
path_loc = f"{reference_output_dir}/members_seed_loc_dict.pickle"
with open(path_LET,'rb') as handle:
    mems_dict = pickle.load(handle)
    random_seeds = np.load(path_seeds)
    with open(path_loc,'rb') as handle:
        seed_loc_dict = pickle.load(handle)

In [5]:
mems_dict

{'CESM': ['001',
  '002',
  '009',
  '010',
  '011',
  '012',
  '013',
  '014',
  '015',
  '016',
  '017',
  '018',
  '020',
  '021',
  '023',
  '024',
  '025',
  '030',
  '031',
  '034',
  '035',
  '101',
  '102',
  '103',
  '104'],
 'GFDL': ['01',
  '02',
  '03',
  '04',
  '05',
  '06',
  '08',
  '09',
  '10',
  '11',
  '12',
  '13',
  '14',
  '16',
  '17',
  '18',
  '19',
  '20',
  '22',
  '23',
  '26',
  '27',
  '28',
  '29',
  '30'],
 'CanESM2': ['r1r10',
  'r1r9',
  'r3r1',
  'r4r5',
  'r5r10',
  'r2r1',
  'r3r2',
  'r3r9',
  'r4r6',
  'r5r2',
  'r1r6',
  'r2r2',
  'r3r4',
  'r4r1',
  'r4r7',
  'r5r4',
  'r1r7',
  'r3r6',
  'r4r8',
  'r5r5',
  'r2r8',
  'r3r7',
  'r4r3',
  'r5r1',
  'r5r9']}

In [ ]:
# Setting the date range to unify the date type
# The LET output ranges from 1982-2017

# Define date range
date_range_start = '1982-01-01T00:00:00.000000000'
date_range_end = '2017-01-01T00:00:00.000000000'

# create date vector
dates = pd.date_range(start=date_range_start, 
                      end=date_range_end,freq='MS') + np.timedelta64(14, 'D')

In [ ]:
# "pre_argo" is a python file with supporting functions
# This creates a dataframe with all inputs (features and target variables) needed for the machine learning

for ens, mem_list in mems_dict.items(): 
    print(ens)
    for member in mem_list:
        print(member)
        
        df = pre_argo.create_inputs(ensemble_dir_head, ens, member, dates)
    
        # Save the data frame 
        pre_argo.save_clean_data(df, data_output_dir, ens, member)
        #del df

### <font color='red'>Section II</font> 
#### For Machine Learning

In [ ]:
# Accessing directories

root_dir = "/local/data/artemis/workspace/theimdal/GO-BGC"

data_output_dir = f"{root_dir}/LET_selected_XGB/500_floats_optimized"

model_output_dir = f"{root_dir}/models/500_floats_optimized/bias4/trained"
recon_output_dir = f"{root_dir}/models/500_floats_optimized/bias4/reconstructions"
other_output_dir = f"{root_dir}/models/500_floats_optimized/bias4/performance_metrics"

approach = 'xg'
approach_output_dir = f"{other_output_dir}/{approach}"

reference_output_dir = "/data/artemis/workspace/theimdal/saildrone/LET_pickle_files"

# Number of cores you have access to for model training

jobs = 30

In [ ]:
# Loading references
path_LET = f"{reference_output_dir}/members_LET_dict.pickle"
path_seeds = f"{reference_output_dir}/random_seeds.npy"
path_loc = f"{reference_output_dir}/members_seed_loc_dict.pickle"

with open(path_LET,'rb') as handle:
    mems_dict = pickle.load(handle)
    
random_seeds = np.load(path_seeds)    
    
with open(path_loc,'rb') as handle:
    seed_loc_dict = pickle.load(handle)

In [6]:
# Load Best Parameters from optimization from previous pCO2-Residual Run
# These parameters were held constant for all sampling experiments so that we could strictly 
# assess the impact of the floats

path_bp="/data/artemis/workspace/vbennington/full_sst/pCO2_DIC/models/performance_metrics/xg/xg_best_params_dict.pickle"
with open(path_bp,'rb') as handle:
    best_params = pickle.load(handle)
print(best_params)

{'CESM': {'max_depth': 6, 'n_estimators': 4000}, 'GFDL': {'max_depth': 6, 'n_estimators': 4000}, 'MPI': {'max_depth': 6, 'n_estimators': 4000}, 'CanESM2': {'max_depth': 6, 'n_estimators': 4000}}


In [ ]:
# Defining inputs for the modeling process

# Train-validate-test split proportions
val_prop = .2
test_prop = .2

# Parameter grids
xg_param_grid = {"n_estimators":[2000, 3000, 4000],
                 "max_depth":[4, 5, 6]
                }
# Feature and target lists for feeding into the models
features_sel = ['sst','sst_anom','sss','sss_anom','mld_clim_log','chl_log','chl_log_anom','xco2','A', 'B', 'C', 'T0', 'T1'] 

# What we reconstruct: pCO2-Residual = pCO2 - pCO2_T
target_sel = ['pCO2_pCO2T_diff'] 

In [ ]:
# We want to train with 4 out of every 5 months, and test on the fifth month
# so that we don't train and test in the same month

date_range_start = '1979-01-01T00:00:00.000000000'
date_range_end = '2017-01-01T00:00:00.000000000'

# Create date vector
dates = pd.date_range(start=date_range_start, 
                      end=date_range_end,freq='MS') + np.timedelta64(14, 'D')
select_dates = []
test_dates = []
for i in range(0,len(dates)):
    if i % 5 != 0:
        select_dates.append(dates[i])
    if i % 5 == 0:
        test_dates.append(dates[i])
year_mon = []
for i in range(0,len(select_dates)):
    tmp = select_dates[i]
    year_mon.append(f"{tmp.year}-{tmp.month}")
test_year_mon = []
for i in range(0,len(test_dates)):
    tmp = test_dates[i]
    test_year_mon.append(f"{tmp.year}-{tmp.month}")

In [ ]:
# Make dictionary to "store" performance metrics for the reconstruction
# "Unseen" = every 1x1 grid cell that is not in the train or test set.
# Since we are using a model testbed and not real-world observations we have unseen output 
test_performance = defaultdict(dict)
unseen_performance = defaultdict(dict)

In [ ]:
K_folds = 3
approach = "xg"
first_mem = False # Switch to True if using gridsearch to find best_params

print(datetime.datetime.now())
for ens, mem_list in mems_dict.items():
    
    for member in mem_list:
        print(ens)
        print(member)
        seed_loc = seed_loc_dict[ens][member] 
        
        
        # Data file path
        data_dir = f"{data_output_dir}/{ens}/member_{member}"
        fname = f"data_clean_2D_mon_{ens}_{member}_1x1_198201-201701.pkl"
        file_path = f"{data_dir}/{fname}"
        
        # Read in data, create some selection filters, produce a reduced dataframe
        df = pd.read_pickle(file_path)
        
        # =====================================================================
        ### THIS CODE IS ONLY NEEDED WHEN ADDING UNCERTAINTY OR BIAS OF FLOATS
        # =====================================================================
              
        #add random error to every float observation
        df['random_number'] = np.random.uniform(-11,11, len(df))
        df['pCO2_pCO2T_diff_new'] = np.where(df.argo_mask == 1, df.pCO2_pCO2T_diff+df.random_number, df.pCO2_pCO2T_diff)
        
        ################ OR ################ 
        
        #add +4 uatm to every float observation
        df['pCO2_pCO2T_diff_new'] = np.where(df.argo_mask == 1, df.pCO2_pCO2T_diff+4, df.pCO2_pCO2T_diff) 
        
        #check the dataframe (adding bias/uncertainty)
        display(df[df.socat_mask==1])
        assert 1==0
        
        # =====================================================================
        ### THIS CODE IS ONLY NEEDED WHEN ADDING UNCERTAINTY OR BIAS OF FLOATS
        # =====================================================================
        
        df['year'] = df.index.get_level_values('time').year
        df['mon'] = df.index.get_level_values('time').month
        df['year_month'] = df['year'].astype(str) + "-" + df['mon'].astype(str)
        
  
        #For pCO2-Residual:
        # Apply mask (remove coast, shallow seas, Arctic etc.) and remove pCO2 values between -250 and +250 ppm
        recon_sel = (~df[features_sel+target_sel+['net_mask']].isna().any(axis=1)) & ((df[target_sel] < 250) & (df[target_sel] > -250)).to_numpy().ravel()
       
        # Select 1x1 grid cells that are not masked as above, and that represents SOCAT and Argo sampling
        sel = (recon_sel & ((df['socat_mask'] == 1) | (df['argo_mask']==1)))  
      
        # Select training set:
        # Need to train where there is SOCAT/Argo data and during a non-test year
        train_sel = (sel & (pd.Series(df['year_month']).isin(year_mon))).to_numpy().ravel()
        print(sum(train_sel))
    
        # Select test set, not during a training month
        test_sel = (sel & (pd.Series(df['year_month']).isin(test_year_mon))).to_numpy().ravel()
        print(sum(test_sel))
    
        # Select "unseen" data - all 1x1 grid cells that are NOT in the train or test set
        unseen_sel = (recon_sel & (df['socat_mask'] == 0) & (np.isnan(df['argo_mask']))) # locations not masked, not ridiculous pCO2, 

        ################################################################################################################################
        
        # Convert dataframe to numpy arrays, train/val/test split
        X = df.loc[sel,features_sel].to_numpy()         
        y = df.loc[sel,target_sel].to_numpy().ravel()
        
        # Convert dataframe to numpy arrays, train/val/test split
        Xtrain = df.loc[train_sel,features_sel].to_numpy() # create Xtrain and Xtest to randomly select from for X_train and X_test
        ytrain = df.loc[train_sel,target_sel].to_numpy().ravel()
        
        N = Xtrain.shape[0]
        train_val_idx, train_idx, val_idx, test_idx = pre_argo.train_val_test_split(N, test_prop, val_prop, random_seeds, seed_loc)
        X_train_val, X_train, X_val, X_test_tmp, y_train_val, y_train, y_val, y_test_tmp = pre_argo.apply_splits(Xtrain, ytrain, train_val_idx, train_idx, val_idx, test_idx) 
        
        # We don't use X_test_tmp or y_test_tmp when test years are used (X_test and y_test set above)
        X_test = df.loc[test_sel,features_sel].to_numpy()                #  Test metrics on all of SOCAT data from test years
        y_test = df.loc[test_sel,target_sel].to_numpy().ravel()    
        
        # Define the model based on which approach to use    
        if first_mem:
            model = XGBRegressor(random_state=random_seeds[4,seed_loc], n_jobs=jobs)
            param_grid = xg_param_grid
            grid = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=K_folds, return_train_score=False, refit=False)
            grid.fit(X_train_val, y_train_val)
            best_params[ens] = grid.best_params_
            print(best_params)
            first_mem = False

        # Fit the model on train/validation data
        model = XGBRegressor(random_state=random_seeds[5,seed_loc], **best_params[ens], n_jobs=jobs)
        model.fit(X_train_val, y_train_val)          

        # Save the model
        pre_argo.save_model(model, model_output_dir, approach, ens, member)   

        # Calculate test error metrics and store in a dictionary
        y_pred_test = model.predict(X_test)
        test_performance[ens][member] = pre_argo.evaluate_test(y_test, y_pred_test)
        print(test_performance[ens][member])
        
        # Redo this analysis on the unseen data
        y_pred_unseen = model.predict(df.loc[unseen_sel,features_sel].to_numpy())
        y_unseen = df.loc[unseen_sel,target_sel].to_numpy().ravel()
        unseen_performance[ens][member] = pre_argo.evaluate_test(y_unseen, y_pred_unseen)
        print(unseen_performance[ens][member])

        # Create the reconstruction and save it
        # "seen": All SOCAT and USV locations for all training years (not test years)
        y_pred_seen = model.predict(X)
        
        # Full reconstruction (all 1x1 degree grid cells)
        df['pCO2_DIC_recon'] = np.nan
        df.loc[unseen_sel,['pCO2_DIC_recon']] = y_pred_unseen   # Not in a SOCAT location, not even in test year
        df.loc[sel,['pCO2_DIC_recon']] = y_pred_seen
        
        # All time/locations not sampled by SOCAT and Argo ("unseen")
        df['pCO2_DIC_nosocat'] = np.nan
        df.loc[unseen_sel,['pCO2_DIC_nosocat']] = y_pred_unseen
        df.loc[sel,['pCO2_DIC_nosocat']] = np.nan
        
        # Only at time/locations of SOCAT and Argo sampling ("training data")
        df['pCO2_DIC_socat'] = np.nan
        df.loc[unseen_sel,['pCO2_DIC_socat']] = np.nan
        df.loc[sel,['pCO2_DIC_socat']] = y_pred_seen
     
        df['pCO2_DIC'] = df['pCO2_pCO2T_diff']
        
        # =========================================
        ### DESCRIPTION OF SAVED DATA SETS
        # =========================================
        
        #pCO2_DIC = pCO2-Residual
        #pCO2_DIC_recon = FULL reconstruction
        #pCO2_DIC_socat = training data (also including Argo locations)
        #pCO2_DIC_nosocat = UNSEEN reconstruction
             
        DS_recon = df[['net_mask','socat_mask','pCO2_DIC','pCO2_DIC_recon','pCO2_DIC_socat','pCO2_DIC_nosocat']].to_xarray()
             
        pre_argo.save_recon(DS_recon, recon_output_dir, approach, ens, member)  

print(datetime.datetime.now())

In [ ]:
# Saving performance metrics (and best parameters if grid search was carried out)

approach_output_dir = f"{other_output_dir}/{approach}"
param_fname = f"{approach_output_dir}/{approach}_best_params_dict.pickle"
test_perform_fname = f"{approach_output_dir}/{approach}_test_performance_dict.pickle"
unseen_perform_fname = f"{approach_output_dir}/{approach}_unseen_performance_dict.pickle"

Path(approach_output_dir).mkdir(parents=True, exist_ok=True)

with open(param_fname, 'wb') as handle:
    pickle.dump(best_params, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(test_perform_fname, 'wb') as handle:
    pickle.dump(test_performance, handle)
with open(unseen_perform_fname, 'wb') as handle:
    pickle.dump(unseen_performance, handle)
    
# Convert performance metrics to dataframes
test_df = pd.DataFrame.from_dict({(i,j): test_performance[i][j]
                                  for i in test_performance.keys()
                                  for j in test_performance[i].keys()},
                                 orient='index')

unseen_df = pd.DataFrame.from_dict({(i,j): unseen_performance[i][j]
                                  for i in unseen_performance.keys()
                                  for j in unseen_performance[i].keys()},
                                 orient='index')

test_df.index.names = ["model","member"]
unseen_df.index.names = ["model","member"]

# Save the dataframes too
test_df_fname = f"{approach_output_dir}/{approach}_test_performance_df.pickle"
unseen_df_fname = f"{approach_output_dir}/{approach}_unseen_performance_df.pickle"

test_df.to_pickle(test_df_fname)
unseen_df.to_pickle(unseen_df_fname)    

### <font color='red'>Section III</font> 
#### Calculate pCO2 from the pCO2-Residual reconstructions

The pCO2-Residual is reconstructed in space and time. pCO2-T is now added back to pCO2-Residual to get pCO2

In [ ]:
# pCO2-Residual reconstruction
run_dir = "/data/artemis/workspace/theimdal/GO-BGC/models/500_floats/uncertainty11/reconstructions/xg"

# Loading references to "pick" members from the LET
# We use 75 members in total, 25 each from CESM, CanESM2 and GFDL
# "mems_dict" is a list of the members of the LET
reference_output_dir = "/data/artemis/workspace/theimdal/saildrone/LET_pickle_files"
path_LET = f"{reference_output_dir}/members_LET_dict.pickle"

with open(path_LET,'rb') as handle:
    mems_dict = pickle.load(handle)

In [ ]:
for ens, mem_list in mems_dict.items():
    print(ens)
    #if ens =="CESM":
        #mem_list = []
        
    for member in mem_list:
        
        member_dir = f"{run_dir}/{ens}/member_{member}" #run_dir = XGB reconstructions
        
        #pCO2-Residual reconstruction
        pco2D_path = f"{member_dir}/xg_recon_pC02_2D_mon_{ens}_{member}_1x1_198201-201701.nc" 
        
        # pCO2-T in this file 
        pco2T_path = f"/data/artemis/workspace/vbennington/LET/{ens}/pCO2_components_{ens}{member}_198201-201701.nc"  
        
        # Path for outputting pCO2 (pCO2-T added back to pCO2-Residual)
        file_out = f"{member_dir}/recon_pCO2DIC_pCO2_2D_mon_{ens}_{member}_1x1_198201-201701.nc"
       
        if ens=="CanESM2":
            pco2T_path = f"/data/artemis/workspace/vbennington/LET/{ens}/pCO2_components_{ens}{member}_198201-201712.nc"  # pCO2-T in this file,
            
        # Load modeled pCO2-T and reconstructed pCO2-DIC:
        pco2T_series = xr.open_dataset(pco2T_path).pCO2_T.transpose("time","ylat","xlon") # pCO2-T calculated from model pCO2 and SST

        pco2D_series = xr.open_dataset(pco2D_path).pCO2_DIC_recon.transpose("time","ylat","xlon") # Reconstructed pCO2-Residual from XGB
        
        pco2D_unseen_series = xr.open_dataset(pco2D_path).pCO2_DIC_nosocat.transpose("time","ylat","xlon") # Reconstructed pCO2-Residual from XGB
        
        
        # Get time coordinate correct
        pco2T_series = pco2T_series.assign_coords({"time":("time",pco2D_series.time)})
        
        pco2_series = pco2T_series + pco2D_series     # Total pCO2 =  pCO2-DIC + pCO2-T
        
        pco2_nosocat =  pco2T_series + pco2D_unseen_series
        
        
        
        comp = xr.Dataset({
                        'pCO2_full_DIC_recon':(["time","ylat","xlon"],pco2D_series),          #full pCO2-Residual reconstruction
                        'pCO2_full_recon':(["time","ylat","xlon"],pco2_series),               #full pCO2 reconstruction
                        'pCO2_DIC_unseen_recon':(["time","ylat","xlon"],pco2D_unseen_series), #unseen pCO2-Residual reconstruction
                        'pCO2_unseen_recon':(["time","ylat","xlon"],pco2_nosocat),            #unseen pCO2 reconstruction
                        'pCO2_T':(["time","ylat","xlon"],pco2T_series)},                      #pCO2T = temp driven component of pCO2
                        coords={'time': (['time'],pco2T_series.time.values),
                        'ylat': (['ylat'],pco2T_series.ylat.values),
                        'xlon':(['xlon'],pco2T_series.xlon.values)})
        
        
        if os.path.isfile(file_out):
            os.remove(file_out)

        comp.to_netcdf(f"{file_out}")
        
        del pco2T_path, pco2D_path, pco2_series, pco2T_series, pco2D_series